## Exercício: Modelo de Linguagem (Bengio 2003) - MLP + Embeddings

Neste exercício iremos treinar uma rede neural similar a do Bengio 2003 para prever a próxima palavra de um texto, data as palavras anteriores como entrada. Esta tarefa é chamada de "Modelagem da Linguagem".

Portanto, você deve implementar o modelo de linguagem inspirado no artigo do Bengio, para prever a próxima palavra usando rede com embeddings e duas camadas.
Sugestão de alguns parâmetros:
* context_size = 9
* max_vocab_size = 3000
* embedding_dim = 64
* usar pontuação no vocabulário
* descartar qualquer contexto ou target que não esteja no vocabulário
* É esperado conseguir uma perplexidade da ordem de 50.
* Procurem fazer asserts para garantir que partes do seu programa estão testadas

Este enunciado não é fixo, podem mudar qualquer um dos parâmetros acima, mas procurem conseguir a perplexidade esperada ou menor.

Gerem alguns frases usando um contexto inicial e depois deslocando o contexto e prevendo a próxima palavra gerando frases compridas para ver se está gerando texto plausível.

Algumas dicas:
- Inclua caracteres de pontuação (ex: `.` e `,`) no vocabulário.
- Deixe tudo como caixa baixa (lower-case).
- A escolha do tamanho do vocabulario é importante: ser for muito grande, fica difícil para o modelo aprender boas representações. Se for muito pequeno, o modelo apenas conseguirá gerar textos simples.
- Remova qualquer exemplo de treino/validação/teste que tenha pelo menos um token desconhecido (ou na entrada ou na saída).
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

Procure por `TODO` para entender onde você precisa inserir o seu código.

## Faz download e carrega o dataset

In [234]:
import os

# Trecho inspirado no trabalho do aluno Elton Cardoso (https://github.com/EltonCN/IA024/blob/main/02-ModeloBengio/Aula_1_2_Exerc%C3%ADcio_Modelo_de_Linguagem_(Bengio_2003)_MLP_%2B_Embeddings.ipynb)

if not os.path.isfile("67724.txt.utf-8"):
    !curl -LO https://www.gutenberg.org/ebooks/67724.txt.utf-8

if not os.path.isfile("67725.txt.utf-8"):
    !curl -LO https://www.gutenberg.org/ebooks/67725.txt.utf-8

In [235]:
text  = open("67724.txt.utf-8","r", encoding="utf8").read()
text += open("67725.txt.utf-8","r", encoding="utf8").read()

paragraphs = text.split("\n\n")
len(paragraphs)

4969

In [236]:
import re

def clean_text(text):
  # Aplicando lower case em todo o texto
  text = text.lower()
  # Removendo quebras de linha
  text = text.replace("\n", " ")
  text = text.replace("\n\n", " ")
  # Removendo caracteres especiais @, #, $, %, _, =, (, )
  text = re.sub(r'[@#$%_=()]', '', text)
  # Substituindo caracteres numéricos por [NUM]
  text = re.sub(r'\d+', '[num]', text)
  # Removendo espaços múltiplos
  text = re.sub(r'\s+', ' ', text)
  return text

cleaned_paragraphs = [clean_text(paragraph) for paragraph in paragraphs if paragraph.strip()]

print(f"""Sample do texto após ser processado: \n
---
{cleaned_paragraphs[150]}
---
""")

print(f"Tamanho da lista de parágrafos processados: {len(cleaned_paragraphs)}")

Sample do texto após ser processado: 

---
--para vós, não duvido; mas isto não é razão de que o seja para outros.
---

Tamanho da lista de parágrafos processados: 4892


## Análise do dataset

In [237]:
# Conta as palavras no dataset
from collections import Counter
import re

def count_words(texts):
    word_counts = Counter()
    for text in texts:
        word_counts.update(re.findall(r'\w+|\S', text.lower())) # Mantém pontuação
    return word_counts

word_counts = count_words(cleaned_paragraphs)

len(word_counts)

12430

## Criando um vocabulário

In [238]:
vocab_size = 3000
most_frequent_words = [word for word, count in word_counts.most_common(vocab_size)]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}
print("Most frequent words:", most_frequent_words[:10])
print("Least frequent words:", most_frequent_words[-10:])

print(len(vocab))

Most frequent words: ['.', ',', '-', 'a', 'que', 'o', 'de', 'e', 'se', ';']
Least frequent words: ['destemido', 'importar', 'mergulhou', 'áquelle', 'morta', 'reflexão', 'hei', 'lh', 'decisivo', 'sobe']
3000


In [239]:
def encode_sentence(sentence, vocab):
    return [vocab.get(word, 0) for word in re.findall(r'\w+|\S', sentence.lower())]

encode_sentence(cleaned_paragraphs[150], vocab)

[3, 3, 19, 230, 2, 13, 0, 10, 40, 118, 13, 38, 278, 7, 5, 6, 400, 19, 203, 1]

## Classe do dataset

In [240]:
%pip install --upgrade torchtext
%pip install --upgrade torch
%pip install --upgrade scikit-learn
%pip install --upgrade numpy

3230.67s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


3237.87s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


3244.99s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


3252.23s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [241]:
import torch
from torch.utils.data import Dataset, DataLoader

class GuaraniDataset(Dataset):
    def __init__(self, text, context_size, vocab):
        self.text = text
        self.context_size = context_size
        self.vocab = vocab
        self.input_target_pairs = self.prepare_dataset()

    def prepare_dataset(self):
        """
        Cria pares de entrada e alvo com base no tamanho da janela de contexto.
        """
        input_target_pairs = []
        for paragraph in self.text:
            tokens = encode_sentence(paragraph, self.vocab)
            if len(tokens) >= self.context_size:
              for i in range(len(tokens) - self.context_size):
                  context = tokens[i:i+self.context_size]
                  target  = tokens[i+self.context_size]
                  input_target_pairs.append((torch.tensor(context), torch.tensor(target)))
        return input_target_pairs

    def __len__(self):
        """
        Retorna o tamanho do dataset.
        """
        return len(self.input_target_pairs)

    def __getitem__(self, idx):
        """
        Retorna um par de entrada e alvo no índice especificado, convertidos em tensores.
        """
        return self.input_target_pairs[idx]

In [242]:
from sklearn.model_selection import train_test_split

context_size = 10
# Dividindo os pares de entrada-alvo em conjuntos de treino e validação
train_data, val_data = train_test_split(
    cleaned_paragraphs,
    test_size=0.2,
    random_state=18)

#Verificando o tamanho dos vetores
print(f"""
---
Conjunto Treinamento: {len(train_data)} [{round((len(train_data) / len(cleaned_paragraphs)) * 100)}%]
---
Conjunto Validação: {len(val_data)} [{round((len(val_data) / len(cleaned_paragraphs)) * 100)}%]
---
""")

train_data = GuaraniDataset(train_data, context_size, vocab)
val_data = GuaraniDataset(val_data, context_size, vocab)


---
Conjunto Treinamento: 3913 [80%]
---
Conjunto Validação: 979 [20%]
---



In [243]:
batch_size   = 30
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=batch_size, shuffle=True)

## Model

In [244]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, hidden_dim):
        """
        Inicializa o modelo de linguagem.
        :param vocab_size: Tamanho do vocabulário.
        :param embedding_dim: Dimensão dos embeddings de palavras.
        :param context_size: Tamanho da janela de contexto (número de palavras de entrada).
        :param hidden_dim: Dimensão da camada oculta.
        """
        super(LanguageModel, self).__init__()

        self.context_size  = context_size
        self.embedding_dim = embedding_dim
        self.hidden_dim    = hidden_dim
        self.vocab_size    = vocab_size

        self.embeddings = nn.Embedding((self.vocab_size + 1), self.embedding_dim)
        self.linear1    = nn.Linear(self.context_size * self.embedding_dim, self.hidden_dim*3)
        self.linear2    = nn.Linear(self.hidden_dim*3, self.hidden_dim*3)
        self.linear3    = nn.Linear(self.hidden_dim*3, (self.vocab_size + 1))
        self.relu       = nn.ReLU()

    def forward(self, inputs):
        """
        Passo forward do modelo.
        """
        # Fase forward para a camada de word features
        o = self.embeddings(inputs).view((-1, self.context_size * self.embedding_dim))
        o = torch.flatten(o, start_dim=1)
        o = self.linear1(o)
        o = self.linear2(o)
        o = self.relu(o)
        return self.linear3(o)
        
    #def forward(self, inputs):
    #    """
    #    Passo forward do modelo.
    #    """
    #    # Fase forward para a camada de word features
    #    o = self.embeddings(inputs)  # Concatena embeddings das palavras de contexto
    #    o = torch.flatten(o, start_dim=1)
    #    # Fase forward para a camada oculta
    #    o = F.tanh(self.linear1(o))
    #    # Fase forward para as unidades de saída
    #    o = self.linear2(o)
    #    log_probs = F.log_softmax(o, dim=1)
    #    return log_probs


In [245]:
embedding_dim = 64                              # Dimensão do embedding -> Dimensão baseada no modelo de embeddings text-embedding-ada-002 da OpenAI
hidden_dim    = embedding_dim * context_size    # Dimensão da camada oculta

# Instância do modelo
model = LanguageModel(vocab_size, embedding_dim, context_size, hidden_dim)

print(model)

LanguageModel(
  (embeddings): Embedding(3001, 64)
  (linear1): Linear(in_features=640, out_features=1920, bias=True)
  (linear2): Linear(in_features=1920, out_features=1920, bias=True)
  (linear3): Linear(in_features=1920, out_features=3001, bias=True)
  (relu): ReLU()
)


In [246]:
sample = next(iter(train_loader))
input = sample[0]
target = sample[1]
output = model(input)

In [247]:
print(f"""
---
{output.argmax(dim=1)}
---
{target}
""")


---
tensor([2183,   63, 2350, 2495, 2731, 2186, 2447, 1198, 1374, 2560,  549, 2852,
        1342,  377, 2596, 1729, 2269,  880, 2491, 1763,   77,    6, 1763, 1342,
         727, 1042,   77,  693,  324, 2495])
---
tensor([ 301,    2,    6,    4,    4, 1762,    5, 2409,  953,   22,   46,  458,
           2,    0,  841,  347,   80,   31,   23,   11,    0,  987,    6,   65,
        1814, 2117,   12,    0,    1,    3])



## Treinamento e Avaliação

In [248]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

LanguageModel(
  (embeddings): Embedding(3001, 64)
  (linear1): Linear(in_features=640, out_features=1920, bias=True)
  (linear2): Linear(in_features=1920, out_features=1920, bias=True)
  (linear3): Linear(in_features=1920, out_features=3001, bias=True)
  (relu): ReLU()
)

In [249]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
import numpy as np


# Definições iniciais
epochs = 10
lr = 0.01 # Defina uma taxa de aprendizado inicial
criterion = nn.CrossEntropyLoss()  # Função de perda de entropia cruzada para classificação
optimizer = optim.SGD(model.parameters(), lr=lr)  # Usando AdamW como otimizador

# Loop de treinamento
for epoch in range(epochs):
    start_time = time.time()  # Start time of the epoch
    model.train()  # Coloca o modelo em modo de treinamento
    running_loss = 0.0
    for inputs, labels in train_loader:  # Supondo que 'train_loader' é seu DataLoader de treino
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)  # Faz a predição
        loss = criterion(outputs, labels)  # Calcula a perda

        optimizer.zero_grad()  # Zera os gradientes acumulados
        loss.backward()  # Propagação reversa
        optimizer.step()  # Atualiza os pesos

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)  # Calcula a loss média por época
    perplexity = np.exp(epoch_loss)  # Calcula a perplexidade para a época

    end_time = time.time()  # End time of the epoch
    epoch_duration = end_time - start_time  # Duration of epoch

    # Avaliação no conjunto de validação
    model.eval()  # Coloca o modelo em modo de avaliação
    val_running_loss = 0.0
    with torch.no_grad():  # Desativa o cálculo de gradientes
        for inputs, labels in val_loader:  # Supondo que 'val_loader' é seu DataLoader de validação
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * inputs.size(0)

    val_epoch_loss = val_running_loss / len(val_loader.dataset)
    val_perplexity = np.exp(val_epoch_loss)  # Calcula a perplexidade para o conjunto de validação

    print(f'Epoch [{epoch+1}/{epochs}], \
        Loss: {loss.item():.4f}, \
        Perplexity: {perplexity:.2f}, \
        Elapsed Time: {epoch_duration:.2f} sec, \
        Validation Loss: {val_epoch_loss:.4f}, \
        Validation Perplexity: {val_perplexity:.2f}')

Epoch [1/10],         Loss: 3.5526,         Perplexity: 282.33,         Elapsed Time: 5.92 sec,         Validation Loss: 5.0972,         Validation Perplexity: 163.56
Epoch [2/10],         Loss: 4.5566,         Perplexity: 167.53,         Elapsed Time: 5.95 sec,         Validation Loss: 4.9437,         Validation Perplexity: 140.28
Epoch [3/10],         Loss: 3.7503,         Perplexity: 133.63,         Elapsed Time: 6.06 sec,         Validation Loss: 4.8328,         Validation Perplexity: 125.57
Epoch [4/10],         Loss: 5.0449,         Perplexity: 110.19,         Elapsed Time: 6.16 sec,         Validation Loss: 4.7476,         Validation Perplexity: 115.31
Epoch [5/10],         Loss: 3.7046,         Perplexity: 91.36,         Elapsed Time: 5.97 sec,         Validation Loss: 4.6759,         Validation Perplexity: 107.33
Epoch [6/10],         Loss: 4.9820,         Perplexity: 75.80,         Elapsed Time: 5.95 sec,         Validation Loss: 4.6300,         Validation Perplexity: 102.51


## Exemplo de uso

In [250]:
#text = ""
#
#def generate_text(model, vocab, text, max_length):
#    """TODO: implemente a função para gerar texto até atingir o max_length"""
#
#context = 5
#max_length= 10
#generate_text(text, max_length)